In [45]:
import pandas as pd
from faker import Faker


In [46]:
customers = pd.read_csv("data/olist_customers_dataset.csv")
orders = pd.read_csv("data/olist_orders_dataset.csv")
order_items = pd.read_csv("data/olist_order_items_dataset.csv")
payments = pd.read_csv("data/olist_order_payments_dataset.csv")
reviews = pd.read_csv("data/olist_order_reviews_dataset.csv")

In [47]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [48]:
customers.shape

(99441, 5)

In [49]:
customers.info

<bound method DataFrame.info of                             customer_id                customer_unique_id  \
0      06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1      18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2      4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3      b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4      4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   
...                                 ...                               ...   
99436  17ddf5dd5d51696bb3d7c6291687be6f  1a29b476fee25c95fbafc67c5ac95cf8   
99437  e7b71a9017aa05c9a7fd292d714858e8  d52a67c98be1cf6a5c84435bd38d095d   
99438  5e28dfe12db7fb50a4b2f691faecea5e  e9f50caf99f032f0bf3c55141f019d99   
99439  56b18e2166679b8a959d72dd06da27f9  73c2643a0a458b49f58cea58833b192e   
99440  274fa6071e5e17fe303b9748641082c8  84732c5050c01db9b23e19ba39899398   

       customer_zip_code_prefix          cu

In [50]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [51]:
items_agg = order_items.groupby("order_id").agg(
    num_items=("order_item_id", "count"),
    total_price=("price", "sum"),
    total_freight=("freight_value", "sum")
).reset_index()

In [52]:
payments_agg = payments.groupby("order_id").agg(
    total_payment=("payment_value", "sum"),
    payment_type=("payment_type", lambda x: x.mode()[0] if not x.mode().empty else "unknown"),
    installments=("payment_installments", "max")
).reset_index()

In [53]:
reviews_sel = reviews[["order_id", "review_score", "review_comment_message"]]

In [54]:
df = (orders
      .merge(customers, on="customer_id", how="left")
      .merge(items_agg, on="order_id", how="left")
      .merge(payments_agg, on="order_id", how="left")
      .merge(reviews_sel, on="order_id", how="left"))

In [55]:
df["purchase_date"] = pd.to_datetime(df["order_purchase_timestamp"])
df["delivered_date"] = pd.to_datetime(df["order_delivered_customer_date"])
df["delivery_time_days"] = (df["delivered_date"] - df["purchase_date"]).dt.days

In [56]:
df["review_comment_message"] = df["review_comment_message"].fillna("No review")

In [57]:
drop_cols = ["customer_zip_code_prefix", "order_approved_at", 
             "order_delivered_carrier_date", "order_estimated_delivery_date",
             "order_purchase_timestamp", "order_delivered_customer_date"]
df = df.drop(columns=drop_cols)

In [58]:
fake = Faker()
unique_customers = df[["customer_unique_id"]].drop_duplicates().reset_index(drop=True)

In [ ]:
unique_customers["customer_name"] = [fake.name() for _ in range(len(unique_customers))]

In [61]:
unique_customers[["first_name", "last_name"]] = (
    unique_customers["customer_name"].str.split(n=1, expand=True)
)

unique_customers["customer_email"] = (
    unique_customers["first_name"].str.lower().str.replace(r"[^a-z]", "", regex=True) + "." +
    unique_customers["last_name"].str.lower().str.replace(r"[^a-z]", "", regex=True) +
    "@mail.com"
)

In [65]:
unique_customers.drop(columns=['customer_name'], inplace=True)

In [66]:
unique_customers

,customer_unique_id,first_name,last_name,customer_email
0,7c396fd4830fd04220f754e42b4e5bff,Johnny,Flowers,johnny.flowers@mail.com
1,af07308b275d755c9edb36a90c618231,Charles,Nguyen,charles.nguyen@mail.com
2,3a653a41f6f9fc3d2a113cf8398680e8,Ronald,Jordan,ronald.jordan@mail.com
3,7c142cf63193a1473d2e66489a9ae977,Krystal,Collier,krystal.collier@mail.com
4,72632f0f9dd73dfee390c9b22eb56dd6,Richard,Villanueva,richard.villanueva@mail.com
...,...,...,...,...
96091,6359f309b166b0196dbf7ad2ac62bb5a,Chad,Harris,chad.harris@mail.com
96092,da62f9e57a76d978d02ab5362c509660,Adam,Pena,adam.pena@mail.com
96093,737520a9aad80b3fbbdad19b66b37b30,Stephanie,Dunn,stephanie.dunn@mail.com
96094,5097a5312c8b157bb7be58ae360ef43c,Alexander,Robinson,alexander.robinson@mail.com


In [67]:
df = df.merge(unique_customers, on="customer_unique_id", how="left")

In [68]:
df = df.drop_duplicates().reset_index(drop=True)


In [69]:
df

,order_id,customer_id,order_status,customer_unique_id,customer_city,customer_state,num_items,total_price,total_freight,total_payment,payment_type,installments,review_score,review_comment_message,purchase_date,delivered_date,delivery_time_days,first_name,last_name,customer_email
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,1.0,29.99,8.72,38.71,voucher,1.0,4.0,"Não testei o produto ainda, mas ele veio corre...",2017-10-02 10:56:33,2017-10-10 21:25:13,8.0,Johnny,Flowers,johnny.flowers@mail.com
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,af07308b275d755c9edb36a90c618231,barreiras,BA,1.0,118.70,22.76,141.46,boleto,1.0,4.0,Muito bom o produto.,2018-07-24 20:41:37,2018-08-07 15:27:45,13.0,Charles,Nguyen,charles.nguyen@mail.com
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,3a653a41f6f9fc3d2a113cf8398680e8,vianopolis,GO,1.0,159.90,19.22,179.12,credit_card,3.0,5.0,No review,2018-08-08 08:38:49,2018-08-17 18:06:29,9.0,Ronald,Jordan,ronald.jordan@mail.com
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,7c142cf63193a1473d2e66489a9ae977,sao goncalo do amarante,RN,1.0,45.00,27.20,72.20,credit_card,1.0,5.0,O produto foi exatamente o que eu esperava e e...,2017-11-18 19:28:06,2017-12-02 00:28:42,13.0,Krystal,Collier,krystal.collier@mail.com
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,72632f0f9dd73dfee390c9b22eb56dd6,santo andre,SP,1.0,19.90,8.72,28.62,credit_card,1.0,5.0,No review,2018-02-13 21:18:39,2018-02-16 18:17:02,2.0,Richard,Villanueva,richard.villanueva@mail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99756,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,6359f309b166b0196dbf7ad2ac62bb5a,sao jose dos campos,SP,1.0,72.00,13.08,85.08,credit_card,3.0,5.0,No review,2017-03-09 09:54:05,2017-03-17 15:08:01,8.0,Chad,Harris,chad.harris@mail.com
99757,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,da62f9e57a76d978d02ab5362c509660,praia grande,SP,1.0,174.90,20.10,195.00,credit_card,3.0,4.0,So uma peça que veio rachado mas tudo bem rs,2018-02-06 12:58:58,2018-02-28 17:37:56,22.0,Adam,Pena,adam.pena@mail.com
99758,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,737520a9aad80b3fbbdad19b66b37b30,nova vicosa,BA,1.0,205.99,65.02,271.01,credit_card,5.0,5.0,Foi entregue antes do prazo.,2017-08-27 14:46:43,2017-09-21 11:24:17,24.0,Stephanie,Dunn,stephanie.dunn@mail.com
99759,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,5097a5312c8b157bb7be58ae360ef43c,japuiba,RJ,2.0,359.98,81.18,441.16,credit_card,4.0,2.0,Foi entregue somente 1. Quero saber do outro p...,2018-01-08 21:28:27,2018-01-25 23:32:54,17.0,Alexander,Robinson,alexander.robinson@mail.com


In [72]:
df.shape

(99761, 20)

In [79]:
# this cell shows the newest orders in the dataset
latest_orders = df[df['order_status']!='canceled'].sort_values("purchase_date", ascending=False).head(10)  # newest 10
latest_orders_display = latest_orders[[
    "order_id","order_status","purchase_date","delivered_date","delivery_time_days",
    "first_name","last_name","customer_email","total_payment","payment_type","review_score"
]]
latest_orders_display


,order_id,order_status,purchase_date,delivered_date,delivery_time_days,first_name,last_name,customer_email,total_payment,payment_type,review_score
62545,54282e97f61c23b78330c15b154c867d,shipped,2018-09-03 09:06:57,NaT,NaN,Stephanie,Sanchez,stephanie.sanchez@mail.com,166.46,voucher,1.0
88384,35a972d7f8436f405b56e36add1a7140,delivered,2018-08-29 15:00:37,2018-08-30 16:23:36,1.0,Heather,Ortega,heather.ortega@mail.com,93.75,credit_card,5.0
69900,03ef5dedbe7492bdae72eec50764c43f,delivered,2018-08-29 14:52:00,2018-08-30 16:36:59,1.0,Lindsey,Watts,lindsey.watts@mail.com,33.23,credit_card,3.0
31580,168626408cb32af0ffaf76711caae1dc,delivered,2018-08-29 14:18:28,2018-08-30 16:52:31,1.0,Jared,Murphy,jared.murphy@mail.com,61.29,voucher,5.0
30166,0b223d92c27432930dfe407c6aea3041,delivered,2018-08-29 14:18:23,2018-08-30 16:24:55,1.0,Dwayne,Doyle,dwayne.doyle@mail.com,510.96,credit_card,5.0
99689,52018484704db3661b98ce838612b507,delivered,2018-08-29 12:25:59,2018-08-30 22:48:27,1.0,Mary,Olson MD,mary.olsonmd@mail.com,73.10,credit_card,1.0
68041,d03ca98f59480e7e76c71fa83ecd8fb6,delivered,2018-08-29 11:06:11,2018-08-30 23:56:54,1.0,Timothy,Scott,timothy.scott@mail.com,119.42,credit_card,5.0
5793,d70442bc5e3cb7438da497cc6a210f80,delivered,2018-08-29 10:22:35,2018-08-30 16:03:19,1.0,Todd,Smith,todd.smith@mail.com,14.29,credit_card,5.0
88700,912859fef5a0bd5059b6d48fa79d121a,delivered,2018-08-29 09:48:09,2018-08-30 23:28:52,1.0,Shannon,Mitchell DDS,shannon.mitchelldds@mail.com,178.25,credit_card,5.0
74997,fb393211459aac00af932cd7ab4fa2cc,delivered,2018-08-29 09:14:11,2018-08-30 13:03:28,1.0,Brenda,Myers,brenda.myers@mail.com,106.95,credit_card,1.0


In [71]:
df[df['order_status'] != 'delivered']

,order_id,customer_id,order_status,customer_unique_id,customer_city,customer_state,num_items,total_price,total_freight,total_payment,payment_type,installments,review_score,review_comment_message,purchase_date,delivered_date,delivery_time_days,first_name,last_name,customer_email
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,36edbb3fb164b1f16485364b6fb04c73,santa rosa,RS,1.0,49.90,16.05,65.95,credit_card,1.0,2.0,fiquei triste por n ter me atendido.,2017-04-11 12:22:08,NaT,NaN,Jeffrey,Williams,jeffrey.williams@mail.com
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,08fb46d35bb3ab4037202c23592d1259,jundiai,SP,1.0,14.49,7.87,22.36,boleto,1.0,1.0,No review,2018-06-04 16:44:48,NaT,NaN,Amanda,Bean,amanda.bean@mail.com
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,c7f8d7b1fffc946d7069574f74c39f4e,santo amaro da imperatriz,SC,1.0,35.00,15.35,50.35,boleto,1.0,3.0,"Gostei do produto, porem fiquei preocupada não...",2018-08-03 17:44:42,NaT,NaN,Benjamin,Baird,benjamin.baird@mail.com
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,9f269af9c49244f6ba4a46985a3cfc2e,sao paulo,SP,1.0,125.90,12.38,138.28,credit_card,2.0,5.0,No review,2017-09-03 14:22:03,NaT,NaN,Nicole,Sanders,nicole.sanders@mail.com
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,528b011eb7fab3d59c336cc7248eed3a,paracatu,MG,1.0,53.99,15.13,69.12,boleto,1.0,NaN,No review,2018-01-10 11:33:07,NaT,NaN,Justin,Patterson,justin.patterson@mail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99603,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,e90598185d2427a35e32ef241a5c04aa,santos,SP,NaN,NaN,NaN,63.89,voucher,1.0,5.0,No review,2018-08-31 16:13:44,NaT,NaN,Kerri,Randall,kerri.randall@mail.com
99633,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,965191786b70912f793e4a777fd623cd,campinas,SP,1.0,129.90,14.66,144.56,boleto,1.0,1.0,"Atendimento horrível, fiquei esperando o produ...",2018-01-05 23:07:24,NaT,NaN,Angela,Mann,angela.mann@mail.com
99667,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,d05c44a138277ad325d915c6b7ccbcdf,sao paulo,SP,NaN,NaN,NaN,107.13,voucher,1.0,4.0,No review,2018-09-06 18:45:47,NaT,NaN,Devin,Davenport,devin.davenport@mail.com
99668,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,e72a90a2b29fe1a8795b284aaaa3246f,rio de janeiro,RJ,NaN,NaN,NaN,195.91,credit_card,2.0,1.0,Fiz a compra do produto para presentear uma pe...,2017-08-23 16:28:04,NaT,NaN,Joseph,Lopez,joseph.lopez@mail.com


In [70]:
df.to_csv("data/olist_cleaned.csv", index=False)